#Brightening the background without hurting the content.

#Sharpen the blurry Text/Image

In [ ]:
import cv2
import numpy as np

# ----------- Sharpen and Enhance the JPG Image -----------
def sharpen_image(image_path, output_path='sharpened_result.jpg'):
    # Load the image
    image = cv2.imread("Paste Path here")

    # Step 1: Apply Gaussian blur (used for sharpening)
    blurred = cv2.GaussianBlur(image, (0, 0), sigmaX=3)

    # Step 2: Sharpen using unsharp masking
    sharpened = cv2.addWeighted(image, 1.7, blurred, -0.7, 0)

    # Step 3: Convert to LAB color space to enhance contrast
    lab = cv2.cvtColor(sharpened, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Apply CLAHE to the L-channel
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)
    merged_lab = cv2.merge((cl, a, b))

    # Convert back to BGR
    final_image = cv2.cvtColor(merged_lab, cv2.COLOR_LAB2BGR)

    # Step 4: Slight denoising (optional but helpful for OCR appearance)
    denoised = cv2.fastNlMeansDenoisingColored(final_image, None, 10, 10, 7, 21)

    # Save the final sharpened image
    cv2.imwrite(output_path, denoised)
    print(f"[+] Sharpened image saved as: {output_path}")

# ----------- Usage -----------
# Replace with your actual JPG filename
sharpen_image("your_result_card.jpg")


#clears the white background Excluding the Image(define coordinates)

In [ ]:
import cv2
import numpy as np

def whiten_background_with_exclusion(image_path, exclusion_coords, output_path='white_bg_excluded.jpg'):
    # Load image
    image = cv2.imread("Paste Path here")
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Adaptive threshold to get foreground (text, images)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY_INV, blockSize=15, C=10)

    # Dilate to cover thicker content
    kernel = np.ones((3, 3), np.uint8)
    foreground_mask = cv2.dilate(thresh, kernel, iterations=1)

    # Invert to get background mask
    background_mask = cv2.bitwise_not(foreground_mask)
    background_mask_3ch = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Replace background with white
    white_bg = np.full_like(image, 255)
    cleaned_image = np.where(background_mask_3ch == 255, white_bg, image)

    # Exclude defined region (paste original content back)
    x, y, w, h = exclusion_coords
    cleaned_image[y:y+h, x:x+w] = image[y:y+h, x:x+w]

    # Save result
    cv2.imwrite(output_path, cleaned_image)
    print(f"[+] White background (with preserved region) saved as: {output_path}")

# ----------- Usage -----------
# Replace with your image path and region to protect (x, y, width, height)
# currently pasted for the test images
whiten_background_with_exclusion("your_result_card.jpg", exclusion_coords=(2059, 0, 391, 574))
